In [1]:
import pandas as pd

In [2]:
import numpy as np
import random
import os
import gc

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(113) # Seed 고정

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv( 'test.csv')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

In [ ]:
def columns_create():    
    global train_lb, test_lb
    # User-ID기준 Rating count & check
    count_columns = []
    check_columns = []
    for i in range(11):
        train_lb[f"count_{i}"] = train_lb[train_lb['Book-Rating'] == i].groupby(['User-ID'])['Book-Rating'].transform('count')
        train_lb[f"count_{i}"] = train_lb[f"count_{i}"].fillna(0)
    
        train_lb[f"check_{i}"] = np.where(train_lb[f"count_{i}"] > 0, 1, 0)
        train_lb = train_lb.drop(columns = [f"count_{i}"]) 
        #count_columns.append(f"count_{i}")
        check_columns.append(f"check_{i}")   
    create_columns = check_columns# count_columns + check_columns
    for i in range(11):
        #train_lb[f"count_{i}"] = train_lb.groupby(['User-ID'])[f"count_{i}"].transform('max')
        train_lb[f"check_{i}"] = train_lb.groupby(['User-ID'])[f"check_{i}"].transform('max')   
    ct_ck_set = train_lb[['User-ID'] + create_columns].drop_duplicates()
    test_lb = pd.merge(test_lb, ct_ck_set, on = 'User-ID', how = 'left')
    test_lb[create_columns] = test_lb[create_columns].fillna(0)    
    return check_columns, train_lb, test_lb #count_columns, 


def feature_engineering(df):
    df['Location'] = [re.sub(r'[^0-9a-zA-Z:,]',  ' ',str(i)) for i in df['Location']]
    
    df['City'] = [(i.split(',')[0]).lstrip().lower() for i in df['Location']]
    df['State'] = [(i.split(',')[1]).lstrip().lower() for i in df['Location']]
    df['Country'] = [(i.split(',')[2]).lstrip().lower() for i in df['Location']]

    labels = ['0-3','3-6','6-8','8-12','12-18','18-25','25-34','35-44','45-54','55-64','65-74','75+']
    bins = [0, 3, 6, 8, 12, 18, 25, 34, 44, 54, 64, 74, 250]
    
    # Age 이상치 처리
    df['Age'] = df['Age'].apply(lambda x: 3 if x<3 else x)
    df['Age'] = df['Age'].apply(lambda x: 100 if x>100 else x)

    #df.loc[(df['Age'] > 90) | (df['Age'] < 3), 'Age'] = np.nan
    
    # 평균값으로 대체
    #df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Age'] = df['Age'].astype(np.int32)

    
    df['Age_gb'] = pd.cut(df.Age, bins, labels = labels,include_lowest = True)
    
    # 출판년도 그룹화
    # 만약 출판연도가 null이 있다면 정보없음(-1)로 채움
    #df['Year-Of-Publication'] = df['Year-Of-Publication'].fillna(-1)
    #labels = ['Unknown', '-1900', '1900-1950', '1950-1960', '1960-1970', '1970-1980', '1980-1990', '1990-2000', '2000-2010', '2010-2020', '2020-']
    #bins = [-1, 0, 1900, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 3000]
    #df['Pub_gb'] = pd.cut(df['Year-Of-Publication'], bins, labels = labels,include_lowest = True)    
    
    return df


In [ ]:
import re 
def preprocessing_data(df):
    
    # preprocessing Location
    # NaN, N/A, etc.. Change 'unknown'
    # Only using Train Data	#
    
    # 최빈값을 사용하기 위해 새로운 데이터 프레임 생성(pd.Series.mode를 이용하면 같은 count수의 값을 list로 묶어서 정확하지 않음)
    new_state = train_lb.groupby(['City'])['State'].value_counts().to_frame().rename(columns = {'State' : 'count'}).reset_index()
    new_state = new_state[(~new_state['City'].isna())&(~new_state['State'].isna())&(new_state['count']!=1)]
    new_state = new_state.sort_values(by=['City', 'count'], ascending=[True, False]).drop_duplicates(subset='City', keep='first')
    new_state = new_state.rename(columns = {'State' : 'N_State'}) 
    new_state = new_state.drop(columns = ['count'])
    
    new_country = train_lb.groupby(['State'])['Country'].value_counts().to_frame().rename(columns = {'Country' : 'count'}).reset_index()
    new_country = new_country[(~new_country['State'].isna())&(~new_country['Country'].isna())&(new_country['count']!=1)]
    new_country = new_country.sort_values(by=['State', 'count'], ascending=[True, False]).drop_duplicates(subset='State', keep='first')
    new_country = new_country.rename(columns = {'Country' : 'N_Country'}) 
    new_country = new_country.drop(columns = ['count'])
    
    df = pd.merge(df, new_country, on = 'State', how = 'left')
    df = pd.merge(df, new_state, on = 'City', how = 'left')
    
    df['Country'] = np.where((df['Country'] == '')|(df['Country'].astype(str) == 'nan'), df['N_Country'], df['Country'])
    df['State'] = np.where((df['State'] == '')|(df['State'].astype(str) == 'nan'), df['N_State'], df['State'])
    df[['Country', 'State', 'City']] = df[['Country', 'State', 'City']].fillna(value= 'Unknown')
    df = df.drop(columns = ['N_Country', 'N_State'])

    # train user list
    # 리뷰 빈도가 단 한 번이거나, train에서 나오지 않은 유저는 'New_user'로 취급함.
    df['User-count'] = df.groupby('User-ID')['User-ID'].transform('count')
    user_list = train['User-ID'].to_numpy()
    df['User-ID'] = np.where(np.logical_or(df['User-count'] == 1, ~df['User-ID'].isin(user_list)), 'New_user', df['User-ID'])
        #df['User-count'] = np.log1p(df['User-count'])
    
    return df
    

In [ ]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder

FEATURE = ['User-ID','Book-Title','Book-Author','Publisher', 'City','State','Country','Age_gb']#', 'Pub_gb']#', 'Language'] ', 

train_lb = train.__deepcopy__() 
test_lb = test.__deepcopy__()

check_columns, train_lb, test_lb = columns_create()

train_lb = feature_engineering(train_lb)
test_lb = feature_engineering(test_lb)

train_lb = preprocessing_data(train_lb)
test_lb = preprocessing_data(test_lb)

train_lb = train_lb.drop(columns = ['Book-ID', 'Location'])
test_lb = test_lb.drop(columns = ['Book-ID', 'Location'])

train_lb[FEATURE] = train_lb[FEATURE].astype(str) 
test_lb[FEATURE] = test_lb[FEATURE].astype(str)


for i in FEATURE:
    le = LabelEncoder()
    le=le.fit(train_lb[i])
    for label in np.unique(test_lb[i].dropna()):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    train_lb[i] = le.transform(train_lb[i])
    test_lb[i] = le.transform(test_lb[i])
    
#for i in FEATURE:
#    # train에는 없고, test에는 있는 원소는 -2 처리
#    oe = OrdinalEncoder(handle_unknown='use_encoded_value',
#                         unknown_value=-2)
#    oe=oe.fit(train_lb[i].to_numpy().reshape(-1, 1))
#    train_lb[i] = oe.transform(train_lb[i].to_numpy().reshape(-1, 1))
#    test_lb[i] = oe.transform(test_lb[i].to_numpy().reshape(-1, 1))

In [ ]:
train_lb.to_csv('train_lb.csv')
test_lb.to_csv('test_lb.csv')

In [7]:
train_lb = pd.read_csv('train_lb.csv')
test_lb = pd.read_csv('test_lb.csv')

In [8]:
X_train = train_lb.drop(columns = ['Unnamed: 0', 'ID', 'Book-Rating', 'Age', 'Year-Of-Publication', 'User-count'])
y_train = train_lb['Book-Rating']
x_test = test_lb.drop(columns = ['Unnamed: 0', 'ID', 'Age', 'Year-Of-Publication','User-count'])

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import MSELoss
from sklearn.model_selection import train_test_split
from tqdm import tqdm

class FeaturesLinear(torch.nn.Module) :
    def __init__(self, field_dims) :
        '''
        Parameter
            field_dims : List of field dimensions
        '''
        super().__init__()
        self.input_dim = sum(field_dims)
        self.linear = nn.Linear(self.input_dim, 1, bias=True)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)

    def forward(self, x):
        '''
        Parameter
            x : Long tensor of size (batch_size, num_fields)
        
        Return
            linear_term : Float tensor of size (batch_size)
        '''
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        sparse_x = torch.zeros(x.size(0), self.input_dim, device=x.device).scatter_(1, x, 1.)
        linear_term = self.linear(sparse_x)
        return linear_term


In [10]:
class FieldAwareEmbedding(torch.nn.Module) :

    def __init__(self, field_dims, embed_dim) :
        '''
        Parameter
            field_dims : List of field dimensions
            embed_dim : Factorization dimension for dense embedding
        '''
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = nn.ModuleList([
            nn.Embedding(feature_size, embed_dim) for feature_size in field_dims
        ])
        for embedding in self.embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)
        
    def forward(self, x) :
        '''
        Parameter
            x : Long tensor of size (batch_size, num_fields)
        
        Return
            dense_x : Long tensor of size (batch_size, num_fields, embed_dim)
        '''
        dense_x = [self.embeddings[i](x[..., i]) for i in range(self.num_fields)]
        dense_x = torch.stack(dense_x, dim=1)
        return dense_x


In [11]:
class FieldAwareFactorizationMachine(torch.nn.Module) :
    def __init__(self, field_dims, embed_dim) :
        '''
        Parameter
            field_dims : List of field dimensions
            embed_dim : Factorization dimension for dense embedding
        '''
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = FieldAwareEmbedding(field_dims, embed_dim)
        self.linear = FeaturesLinear(field_dims)
        
    def square(self, x):
        return torch.pow(x,2)

    def forward(self, x) :
        '''
        Parameter
            x : Long tensor of size (batch_size, num_fields)
        
        Return
            y_ffm : Float tensor of size (batch_size)
        '''
        linear_term = self.linear(x)
        
        dense_x = self.embeddings(x)

        square_of_sum = self.square(torch.sum(dense_x, dim=1))
        sum_of_square = torch.sum(self.square(dense_x), dim=1)
        pairwise_term = 0.5 * torch.sum(square_of_sum - sum_of_square, dim=1)
        
        y_ffm = linear_term.squeeze(1) + pairwise_term

        return y_ffm


In [12]:
X_tr, X_valid, y_tr, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42,
                                                      shuffle=True, stratify=y_train)


In [13]:
train_dataset = TensorDataset(torch.LongTensor(X_tr.to_numpy()),
                              torch.LongTensor(y_tr.to_numpy()))
valid_dataset = TensorDataset(torch.LongTensor(X_valid.to_numpy()),
                              torch.LongTensor(y_valid.to_numpy()))

test_dataset = TensorDataset(torch.LongTensor(x_test.to_numpy()),
                              torch.LongTensor(np.zeros(x_test.shape[0])))

In [14]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.eps = 1e-6
    def forward(self, x, y):
        criterion = MSELoss()
        loss = torch.sqrt(criterion(x, y)+self.eps)
        return loss


In [15]:
def model_train(model, dataloader, loss_fn, optimizer, device) :
    model.train()
    total_loss = 0
    for features, targets in tqdm(dataloader) :
        features = features.to(device)
        targets = targets.to(torch.float32).to(device)
        
        outputs = model(features)
        loss = loss_fn(targets, outputs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss
        
    return (total_loss/len(dataloader)).item()


In [16]:
def model_eval(model, dataloader, loss_fn, device) :
    with torch.no_grad() :
        total_loss = 0
        model.eval()
        for features, targets in tqdm(dataloader) :
            features = features.to(device)
            targets = targets.to(torch.float32).to(device)
            
            outputs = model(features)
            loss = loss_fn(targets, outputs)
            total_loss += loss
            
    return (total_loss/len(dataloader)).item()


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [18]:
features = X_train.columns
idx = {feature:None for feature in features}
for feature in features :
    feature2idx = {v:k for k,v in enumerate(X_train[feature].unique())}
    idx[feature] = len(feature2idx)
    X_train[feature] = X_train[feature].map(feature2idx)


In [19]:
field_dims = np.array(list(idx.values()), dtype=np.uint32)
print(field_dims)

[ 36989 217829  92635  15505      2      2      2      2      2      2
      2      2      2      2      2  13741   1766    349     12]


In [20]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

embed_dim = 4
batch_size = 256
lr = 0.001
epochs = 30

In [ ]:
ffm_model = FieldAwareFactorizationMachine(field_dims, embed_dim=embed_dim)
ffm_model.to(device)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

loss_fn = RMSELoss()
optimizer = torch.optim.Adam(ffm_model.parameters(), lr=lr)

best_loss = int(1e8)
check_cnt = 0

for epoch in range(epochs) :
    train_loss = model_train(ffm_model, train_loader, loss_fn, optimizer, device)
    valid_loss = model_eval(ffm_model, valid_loader, loss_fn, device)
    
    print(f"Epoch {epoch+1}")
    print(f"Train Loss : {train_loss:.2f}, Validation Loss : {valid_loss:.2f}")
    
    if best_loss > valid_loss :
        best_loss = valid_loss
        torch.save(ffm_model.state_dict(), 'ffm_model.pt')
        check_cnt = 0
    else :
        check_cnt += 1
        if check_cnt == 5 :
            print("Early Stopped")
            break

In [21]:
best_model = FieldAwareFactorizationMachine(field_dims, embed_dim).to(device)
best_model.load_state_dict(torch.load('ffm_model.pt'))
best_model = best_model.to(device)

In [22]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
def model_pred(model, dataloader, device) :
    pred = []
    with torch.no_grad() :
        model.eval()
        for features, targets in tqdm(dataloader) :
            features = features.to(device)
            
            outputs = model(features)
            pred.append(outputs)
    return pred

In [24]:
pred = model_pred(best_model, test_loader, device)

  0%|                                                                                           | 0/624 [00:00<?, ?it/s]../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [7,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [7,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [7,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [7,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [7,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [7,0,0], thread: [69,0,0] Assertion `srcIndex < 

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
